In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import pandas as pd
import os
import string
import random

In [16]:
#Hyperparams
lr = 1
gamma = 0.95
embed_size = 256
hidden_size = 512

In [17]:
vocab = getVocab('text.txt')

In [18]:
vocab_size = vocab.size()

In [19]:
model = AttentionModel(embed_size, hidden_size, vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [20]:
corr_sentences = pd.read_csv("data/qs.csv")["Correct Answer"]
corr_sentences = [cleanText(sentence) for sentence in corr_sentences]
corr_tensors = [torch.Tensor(vocab.getSentenceArray(sentence)) for sentence in corr_sentences]

In [21]:
test_sentences = pd.read_csv("data/ratings.csv")
test_sentences["Answer"] = [cleanText(s) for s in test_sentences["Answer"]]
orig_len = len(test_sentences["Answer"])
curr_len = orig_len

for t in range(len(corr_sentences)):
    for i in range(orig_len):
        if t == test_sentences["Question Id"][i]:
            continue
        test_sentences.loc[curr_len] = [t, test_sentences["Answer"][i], 0]
        curr_len += 1

junk_sentences = pd.read_csv("data/wat.csv")
junk_sentences["Answer"] = [cleanText(sentence) for sentence in junk_sentences["Answer"]]

for t in range(len(corr_sentences)):
    for i in range(len(junk_sentences)):
        test_sentences.loc[curr_len] = [t, junk_sentences["Answer"][i], junk_sentences["Rating"][i]]
        curr_len += 1

test_tensors = [torch.tensor(vocab.getSentenceArray(s)) for s in test_sentences["Answer"]]

In [22]:
for e in range(100):
    total_loss = 0
    for t in range(len(test_tensors)):
        curr_a = random.randint(0, len(test_tensors)-1) 
        curr_q = test_sentences["Question Id"][curr_a]
        
        optimizer.zero_grad()
        
        pred = model(corr_tensors[curr_q].long(), test_tensors[curr_a].long()).view(1)
        y = torch.tensor(test_sentences["Rating"][curr_a]).float().view(1)
        loss = criterion(pred, y)
        total_loss += loss.item()
    
        loss.backward()
        optimizer.step()
    
    for param_group in optimizer.param_groups:
        param_group['lr'] *= gamma
        
    print("Epoch:", e, "Loss:", total_loss)

/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0 Loss: 7.208067096100422
Epoch: 1 Loss: 2.3477790712789144
Epoch: 2 Loss: 2.0201978877786964
Epoch: 3 Loss: 0.45836878943720216
Epoch: 4 Loss: 0.9588308176125836
Epoch: 5 Loss: 0.8678273036152128
Epoch: 6 Loss: 0.5046980311352244
Epoch: 7 Loss: 0.4290146143901121
Epoch: 8 Loss: 0.22037911791845932
Epoch: 9 Loss: 0.42299654405423937
Epoch: 10 Loss: 0.31762334021450567
Epoch: 11 Loss: 0.17638741635291266
Epoch: 12 Loss: 0.10876714583976055
Epoch: 13 Loss: 0.09312791810757926
Epoch: 14 Loss: 0.13325239164253144
Epoch: 15 Loss: 0.1844676444470854
Epoch: 16 Loss: 0.16262580969351914
Epoch: 17 Loss: 0.17011783394080893
Epoch: 18 Loss: 0.1278219599132031
Epoch: 19 Loss: 0.13400344196672886
Epoch: 20 Loss: 0.10289015082185404
Epoch: 21 Loss: 0.10528241058117305
Epoch: 22 Loss: 0.055690942236058305
Epoch: 23 Loss: 0.0372847970136313
Epoch: 24 Loss: 0.07045631022822674
Epoch: 25 Loss: 0.11170079905184316
Epoch: 26 Loss: 0.018662667857845983
Epoch: 27 Loss: 0.04454281843790686
Epoch: 28 L

In [23]:
torch.save(model, "saved_models/model2.pt")

In [24]:
test_sentences.head

<bound method NDFrame.head of     Question Id                                             Answer  Rating
0             0             encapsulation polymorphism inheritance     0.7
1             0  the main features of oop are encapsulation pol...     1.0
2             1  the main advantage of oop is better manageable...     1.0
3             1  the main advantage of oop is better manageable...     0.3
4             2  encapsulation is referred to one of the follow...     1.0
5             2             it is data hiding and bundling of data     0.5
6             3  polymorphism means that some code or operation...     1.0
7             3  it is a feature that allows a single object to...     0.9
8             4  a class is based on another class and uses dat...     1.0
9             4           the purpose of inheritance is code reuse     0.2
10            5  it stands for java development kit it is the t...     1.0
11            5                 it stands for java development kit    